In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def polynomial_sframe(feature,degree):
    poly_frame=pd.DataFrame()
    poly_frame['power_1']=feature
    for power in range(2,degree+1):
        name='power_'+str(power)
        poly_frame[name]=feature.apply(lambda x: x**power)
    return(poly_frame)

In [3]:
train_valid_shuffled = pd.read_csv('wk3_kc_house_train_valid_shuffled.csv')

In [4]:
train_valid_shuffled['sqft_living']=train_valid_shuffled['sqft_living'].astype(float)
train_valid_shuffled=train_valid_shuffled.sort('sqft_living')

In [5]:
valid15_shuffled=polynomial_sframe(train_valid_shuffled['sqft_living'],15)

In [6]:
def get_residual_sum_of_squares(model, f, output):
    """
    Purpose: Compute Residual Sum of Squares (RSS)
    Input  : features (x), output (y) and estimated model 
    Output : Residual sum of squares = sum((actual output (y) - predicted output)^2)
    """
    RSS = np.sum((output -f)** 2)
    return(RSS)

In [69]:
def k_fold_cross_validation(k,penalties,data, output):
    m={}
    for penalty in penalties:
        print penalty
        l2_penalty=penalty
        names='rss_dict_'+str(penalty)
        names={}
        n = len(data)
        validation=0.0
        for i in range(k):
            start =(n*i)/k
            end = (n*(i+1))/k-1
            x=data[0:start].append(data[end+1:n])
            y=output[0:start].append(output[end+1:n])
            model=Ridge(alpha=l2_penalty,normalize=True)
            model.fit(x,y)
            name='rss_'+str(i)
            #rss=(metrics.mean_squared_error(output[start:end+1],model.predict(data[start:end+1])))*(n/k)
            rss=get_residual_sum_of_squares(model, model.predict(data[start:end+1]),output[start:end+1])
            validation+=rss
        #return(validation)
            names[name]=rss
        m[str(penalty)]=names
    return(m)

In [60]:
from sklearn.linear_model import Ridge
from sklearn import metrics

In [61]:
penalties=np.logspace(3, 9, num=13)

In [68]:
penalties[1]

3162.2776601683795

In [70]:
d=k_fold_cross_validation(10,penalties, valid15_shuffled, train_valid_shuffled['price'])

1000.0
3162.27766017
10000.0
31622.7766017
100000.0
316227.766017
1000000.0
3162277.66017
10000000.0
31622776.6017
100000000.0
316227766.017
1000000000.0


In [71]:
d

{'1000.0': {'rss_0': 157039434848896.75,
  'rss_1': 117628798362295.94,
  'rss_2': 98233206898076.2,
  'rss_3': 83780534104339.36,
  'rss_4': 78273280760867.9,
  'rss_5': 76526635226669.7,
  'rss_6': 81185387088474.08,
  'rss_7': 120690217136337.36,
  'rss_8': 261227794967515.72,
  'rss_9': 2.6799981220621565e+24},
 '10000.0': {'rss_0': 157730911298608.2,
  'rss_1': 118066421793945.23,
  'rss_2': 98542509343318.73,
  'rss_3': 83968527415158.33,
  'rss_4': 78391537712804.8,
  'rss_5': 76564674718601.97,
  'rss_6': 81189048998797.08,
  'rss_7': 120749332009225.39,
  'rss_8': 261660447908666.16,
  'rss_9': 2.760118120647343e+22},
 '100000.0': {'rss_0': 157800433548127.6,
  'rss_1': 118110425727168.44,
  'rss_2': 98573611471352.22,
  'rss_3': 83987435076931.92,
  'rss_4': 78403431663549.38,
  'rss_5': 76568502905258.14,
  'rss_6': 81189417490719.5,
  'rss_7': 120755268983754.72,
  'rss_8': 261703925274319.5,
  'rss_9': 2.76735229010177e+20},
 '1000000.0': {'rss_0': 157807389546811.97,
  'r

In [64]:
penalty_dataframe=pd.DataFrame(d)

In [65]:
penalty_dataframe

,1000.0,10000.0,100000.0,1000000.0,10000000.0,100000000.0,1000000000.0,3162.27766017,31622.7766017,316227.766017,3162277.66017,31622776.6017,316227766.017
rss_0,1.570394e+14,1.577309e+14,1.578004e+14,1.578074e+14,1.578081e+14,1.578082e+14,1.578082e+14,1.575642e+14,1.577837e+14,1.578057e+14,1.578079e+14,1.578081e+14,1.578082e+14
rss_1,1.176288e+14,1.180664e+14,1.181104e+14,1.181148e+14,1.181153e+14,1.181153e+14,1.181153e+14,1.179609e+14,1.180998e+14,1.181138e+14,1.181152e+14,1.181153e+14,1.181153e+14
rss_2,9.823321e+13,9.854251e+13,9.857361e+13,9.857672e+13,9.857703e+13,9.857707e+13,9.857707e+13,9.846791e+13,9.856614e+13,9.857598e+13,9.857696e+13,9.857706e+13,9.857707e+13
rss_3,8.378053e+13,8.396853e+13,8.398744e+13,8.398933e+13,8.398952e+13,8.398954e+13,8.398954e+13,8.392318e+13,8.398289e+13,8.398887e+13,8.398947e+13,8.398953e+13,8.398954e+13
rss_4,7.827328e+13,7.839154e+13,7.840343e+13,7.840462e+13,7.840474e+13,7.840475e+13,7.840475e+13,7.836301e+13,7.840057e+13,7.840434e+13,7.840471e+13,7.840475e+13,7.840475e+13
rss_5,7.652664e+13,7.656467e+13,7.656850e+13,7.656889e+13,7.656892e+13,7.656893e+13,7.656893e+13,7.655550e+13,7.656758e+13,7.656879e+13,7.656892e+13,7.656893e+13,7.656893e+13
rss_6,8.118539e+13,8.118905e+13,8.118942e+13,8.118945e+13,8.118946e+13,8.118946e+13,8.118946e+13,8.118817e+13,8.118933e+13,8.118945e+13,8.118946e+13,8.118946e+13,8.118946e+13
rss_7,1.206902e+14,1.207493e+14,1.207553e+14,1.207559e+14,1.207559e+14,1.207559e+14,1.207559e+14,1.207351e+14,1.207538e+14,1.207557e+14,1.207559e+14,1.207559e+14,1.207559e+14
rss_8,2.612278e+14,2.616604e+14,2.617039e+14,2.617083e+14,2.617087e+14,2.617088e+14,2.617088e+14,2.615562e+14,2.616935e+14,2.617072e+14,2.617086e+14,2.617087e+14,2.617088e+14
rss_9,2.679998e+24,2.760118e+22,2.767352e+20,2.760172e+18,2.843911e+16,2.004795e+15,1.828835e+15,2.740629e+23,2.766109e+21,2.765782e+19,2.753173e+17,4.261257e+15,1.830165e+15


In [66]:
(penalty_dataframe.sum(axis=0))/10

1000.0           2.679998e+23
10000.0          2.760118e+21
100000.0         2.767363e+19
1000000.0        2.761249e+17
10000000.0       2.951622e+15
100000000.0      3.081913e+14
1000000000.0     2.905953e+14
3162.27766017    2.740629e+22
31622.7766017    2.766110e+20
316227.766017    2.765890e+18
3162277.66017    2.763945e+16
31622776.6017    5.338375e+14
316227766.017    2.907283e+14
dtype: float64